In [27]:
import numpy as np
path = '../data/DonaSet/points/000007.bin'

fields = 4
# Read the binary file into a NumPy array
data = np.fromfile(path, dtype=np.float32)

# Reshape the 1D array to a 2D array with 4 columns
data = data.reshape(-1, 4)
data[0]

array([ 24.313847 , -18.891674 ,   1.3315092,   0.       ], dtype=float32)

In [9]:
import open3d as o3d
path = '/home/riccardo/SUSTechPOINTS/data/234184_2023-01-04_10-08_episodes/lidar/1672856106850014000.pcd'
pcd = o3d.io.read_point_cloud(path)
points = np.array(pcd.points)
points.shape

(26032, 3)

In [19]:
x = data[:,0]
y = data[:,1]
z = data[:,2]

In [21]:
import open3d as o3d
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(data[:, :3]) 
o3d.visualization.draw_geometries([point_cloud])

In [3]:
import numpy as np
# Specify the file path
file_path = 'data/kitti/example/calib/000006.txt'

# Read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Extract Tr_velo_to_cam matrix
for line in lines:
    if line.startswith('Tr_velo_to_cam:'):
        # Extract the matrix values and convert them to a NumPy array
        tr_velo_to_cam_values = [float(x) for x in line.strip().split()[1:]]
        tr_velo_to_cam_values.extend([0,0,0,1])
        tr_velo_to_cam = np.array(tr_velo_to_cam_values).reshape(4, 4)
        break

print("Tr_velo_to_cam matrix:")
print(tr_velo_to_cam)
tr_cam_to_velo = np.linalg.inv(tr_velo_to_cam)

Tr_velo_to_cam matrix:
[[ 0.00775545 -0.9999694  -0.0010143  -0.00727554]
 [ 0.00229406  0.00103212 -0.9999968  -0.06324057]
 [ 0.9999673   0.0077531   0.00230199 -0.2670414 ]
 [ 0.          0.          0.          1.        ]]


In [35]:
from mmdet3d.structures import LiDARInstance3DBoxes
from mmdet3d.structures import CameraInstance3DBoxes
import torch

x_c = np.array([-2.72, 0.82, 48.22, 1])
x_v = tr_cam_to_velo @ x_c.T
x_v, x_c
def convert_box_cam_to_lidar(box,tr_cam_to_velo):
    b =  np.array(box)[0]
    P_c = np.concatenate((b[:3], [1]))
    P_l = tr_cam_to_velo @ P_c.T
    arr = np.concatenate((P_l[:3],[b[5]],[b[3]],[b[4]],[-1.57-b[6] ]))    
    print(arr)
    box_l = LiDARInstance3DBoxes(
        torch.tensor([arr])
    )
    return box_l

box_c = CameraInstance3DBoxes(
     torch.tensor([[-2.72, 0.82, 48.22 ,1.48, 1.56, 3.62, -1.62]])
)
box_l =convert_box_cam_to_lidar(box_c,tr_cam_to_velo)
# box_l,box_c

[48.46644438  3.08947793 -0.76886957  3.61999989  1.55999994  1.48000002
  0.05      ]


In [36]:
import torch
import numpy as np

from mmdet3d.visualization import Det3DLocalVisualizer
from mmdet3d.structures import LiDARInstance3DBoxes
from mmdet3d.structures import CameraInstance3DBoxes

from mmdet3d.structures import (BaseInstance3DBoxes, Box3DMode,
                                CameraInstance3DBoxes, Coord3DMode,
                                DepthInstance3DBoxes, DepthPoints,
                                Det3DDataSample, LiDARInstance3DBoxes,
                                PointData, points_cam2img)

points = np.fromfile('data/kitti/example/000006.bin', dtype=np.float32)
points = points.reshape(-1, 4)
visualizer = Det3DLocalVisualizer()
# set point cloud in visualizer
visualizer.set_points(points)
# Car 0.00 2 -1.55 548.00 171.33 572.40 194.42 1.48 1.56 3.62 -2.72 0.82 48.22 -1.62
# Car 0.00 0 -1.21 505.25 168.37 575.44 209.18 1.67 1.64 4.32 -2.61 1.13 31.73 -1.30
# Car 0.00 0 0.15 49.70 185.65 227.42 246.96 1.50 1.62 3.88 -12.54 1.64 19.72 -0.42

bboxes_3d = CameraInstance3DBoxes(
     torch.tensor([[-12.54, 1.64 ,19.72,1.50 ,1.62 ,3.88,(-0.42)]])
     # torch.tensor([[-2.72,0.82 ,48.22, 1.48 ,1.56 ,3.62, -1.62]])
     # torch.tensor([[-2.61, 1.13 ,31.73,4.32 ,1.64 ,1.67,-1.30]])
)
bboxes_3d_l_manual =  convert_box_cam_to_lidar(bboxes_3d,tr_cam_to_velo)
bboxes_3d_l = bboxes_3d.convert_to(Box3DMode.LIDAR,
                         tr_cam_to_velo)
print(bboxes_3d_l)
# Draw 3D bboxes
visualizer.draw_bboxes_3d(bboxes_3d_l_manual,bbox_color=(255,128,64),rot_axis=2)
visualizer.show()

[19.89309735 12.68906065 -1.64451328  3.88000011  1.62        1.5
 -1.15000001]
LiDARInstance3DBoxes(
    tensor([[19.8931, 12.6891, -1.6445,  1.5000,  3.8800,  1.6200, -1.1508]]))
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


SystemExit: 0

In [17]:
import pickle
file = "../data/KITTI_reduced_noImages/donaset_infos_train.pkl"
with open(file, 'rb') as f:
    info = pickle.load(f)
print(info)

{'metainfo': {'categories': {'Pedestrian': 0, 'Cyclist': 1, 'Car': 2, 'Van': 3, 'Truck': 4, 'Person_sitting': 5, 'Tram': 6, 'Misc': 7, 'DontCare': -1}, 'dataset': 'kitti', 'info_version': '1.1'}, 'data_list': [{'sample_idx': 0, 'images': {'CAM0': {'cam2img': [[707.0493, 0.0, 604.0814, 0.0], [0.0, 707.0493, 180.5066, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]], 'lidar2img': [[602.9436979403572, -707.9132863314612, -12.274842681831494, -216.70101629403663], [176.77725024402153, 8.80879965630006, -707.9361204188122, -102.22321541671865], [0.9999848008155823, -0.0015282672829926014, -0.005290712229907513, -0.33254897594451904], [0.0, 0.0, 0.0, 1.0]]}, 'CAM1': {'cam2img': [[707.0493, 0.0, 604.0814, -379.7842], [0.0, 707.0493, 180.5066, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]], 'lidar2img': [[602.9436979403572, -707.9132863314612, -12.274842681831494, -596.4852162940366], [176.77725024402153, 8.80879965630006, -707.9361204188122, -102.22321541671865], [0.9999848008155823, -0.0

Substitute the labels in Camera Coords into Lidar Coords

In [30]:
def extractCam2LidarMatrix(filename):
    # Read the file
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Extract Tr_velo_to_cam matrix
    for line in lines:
        if line.startswith('Tr_velo_to_cam:'):
            # Extract the matrix values and convert them to a NumPy array
            tr_velo_to_cam_values = [float(x) for x in line.strip().split()[1:]]
            tr_velo_to_cam_values.extend([0,0,0,1])
            tr_velo_to_cam = np.array(tr_velo_to_cam_values).reshape(4, 4)
            break
    return np.linalg.inv(tr_velo_to_cam)
def convert_KITTI_label_to_lidar(filename,cam2lidar):
    # Read the file
    with open(filename, 'r') as file:
        lines = file.readlines()

    new_lines = []
    # Extract Tr_velo_to_cam matrix
    for line in lines:
        arr = line.split(' ')
        type =  arr[0]
        b = np.array(arr[1:]).astype(float)
        P_c = np.concatenate((b[10:13], [1]))
        P_l = cam2lidar @ P_c.T
        tmp = np.concatenate((P_l[:3],[b[9]],[b[7]],[b[8]],[-1.57-b[13] ]))    
        tmp = np.round(tmp,2)
        new_line = type + " " +" ".join(tmp.astype(str))
        new_lines.append(new_line)
    return new_lines
def writeLabelFile(filename, lines):
    with open(filename,"w") as f:
        for line in lines:
            f.write(line + '\n')

In [36]:
import os
import numpy as np
dataset_folder = "../data/KITTI_reduced_lidar/training"
label_folder = os.path.join(dataset_folder,"label_2")
calib_folder = os.path.join(dataset_folder,"calib")
labels_txts = os.listdir(label_folder)
label_dest_folder = os.path.join(dataset_folder,"label_2_new")
if not os.path.exists(label_dest_folder):
    os.makedirs(label_dest_folder)

for label_txt in labels_txts:
    print(label_txt)
    cam2lidar = extractCam2LidarMatrix(os.path.join(calib_folder,label_txt))
    new_lines = convert_KITTI_label_to_lidar(os.path.join(label_folder,label_txt),cam2lidar)
    writeLabelFile(os.path.join(label_dest_folder,label_txt),new_lines)
    

000037.txt
000114.txt
000014.txt
000160.txt
000194.txt
000157.txt
000119.txt
000041.txt
000019.txt
000056.txt
000179.txt
000055.txt
000057.txt
000033.txt
000120.txt
000190.txt
000106.txt
000066.txt
000090.txt
000159.txt
000086.txt
000013.txt
000141.txt
000067.txt
000110.txt
000176.txt
000093.txt
000116.txt
000028.txt
000073.txt
000167.txt
000027.txt
000031.txt
000128.txt
000016.txt
000130.txt
000020.txt
000008.txt
000034.txt
000118.txt
000079.txt
000001.txt
000115.txt
000083.txt
000039.txt
000050.txt
000143.txt
000065.txt
000147.txt
000108.txt
000101.txt
000134.txt
000060.txt
000003.txt
000051.txt
000024.txt
000107.txt
000075.txt
000095.txt
000158.txt
000018.txt
000097.txt
000174.txt
000148.txt
000195.txt
000080.txt
000185.txt
000064.txt
000085.txt
000136.txt
000077.txt
000068.txt
000102.txt
000161.txt
000189.txt
000040.txt
000127.txt
000165.txt
000070.txt
000078.txt
000036.txt
000012.txt
000074.txt
000132.txt
000053.txt
000163.txt
000187.txt
000010.txt
000168.txt
000048.txt
000126.txt

In [20]:
new_lines

['Car 19.71 0.59 -1.43 3.84 1.6 1.76 0.02',
 'Truck 10.51 3.7 -1.59 16.44 4.05 2.5 0.02',
 'Car 24.75 3.64 -1.36 4.39 1.3 1.61 0.03']